In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
source = requests.get(url).text
soup= BeautifulSoup(source, "lxml")
table = soup.find('table')

In [13]:
df=pd.read_html(str(table))[0]

Drop all rows that have Not assigned for both the Borough and the Neighbourhood columns.

In [5]:
rowdrop = df[(df['Borough']=='Not assigned')&(df['Neighbourhood']=='Not assigned')].index
df.drop(rowdrop, inplace=True)

Change all not assigned values to the name of the Borough

In [7]:
df.loc[df['Neighbourhood']==('Not assigned'),'Neighbourhood']=df['Borough']

Merge all postal codes that have two neighbourhoods

In [9]:
df=df.groupby(['Postcode','Borough'],sort=False).agg(lambda x:','.join(x))

In [10]:
df.reset_index(inplace=True)

In [11]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [12]:
df.shape

(103, 3)